<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Torrent_Downloader_%5BStable%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ⚙️ Step 1: Install & Setup (Run Once)
import os
import sys
import time
import requests
import subprocess
from google.colab import drive
from google.colab import files
from IPython.display import clear_output

def install_dependencies():
    try:
        subprocess.run(["aria2c", "--version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        print("✅ Aria2 is already installed.")
    except FileNotFoundError:
        print("⚙️ Installing Aria2 (Clean CLI tool)...")
        subprocess.run(["apt-get", "install", "-y", "aria2"], check=True)
        print("✅ Installation complete.")

def get_trackers():
    print("📡 Fetching latest trackers to boost speed...")
    try:
        url = "https://raw.githubusercontent.com/ngosang/trackerslist/refs/heads/master/trackers_all.txt"
        response = requests.get(url)
        trackers = response.text.replace('\n\n', ',').replace('\n', ',').strip(',')
        print(f"✅ Loaded {len(trackers.split(','))} trackers.")
        return trackers
    except Exception as e:
        print(f"⚠️ Failed to fetch trackers: {e}")
        return ""

install_dependencies()

In [ ]:
# @title 🚀 Step 2: Start Download
# @markdown ### Configuration
use_google_drive = True # @param {type:"boolean"}
source_type = "Magnet" # @param ["Magnet", "File"]
# @markdown *If using File, leave the Magnet Link blank.*
magnet_link = "" # @param {type:"string"}

def run_downloader():
    # --- PATCH 1: STORAGE WARNING ---
    base_dir = "/content/downloads"
    if use_google_drive:
        if not os.path.exists('/content/drive'):
            print("📂 Mounting Google Drive...")
            drive.mount('/content/drive')
        base_dir = "/content/drive/MyDrive/Torrents"
    else:
        # ANSI Escape codes for Red Text
        print("\033[91m" + "⚠️ WARNING: Google Drive is NOT enabled." + "\033[0m")
        print("\033[91m" + "   Files are saved to Temporary Colab Storage." + "\033[0m")
        print("\033[91m" + "   They WILL BE DELETED when this session ends." + "\033[0m")
        time.sleep(2) # Give user time to read

    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # --- PATCH 2: INPUT LOGIC ---
    target_uri = ""

    if source_type == "File":
        if magnet_link:
            print("ℹ️ Note: You selected 'File' mode. The text in 'Magnet Link' will be ignored.")

        print("📂 Please upload your .torrent file now:")
        uploaded = files.upload()
        for filename in uploaded.keys():
            target_uri = os.path.abspath(filename)
            print(f"✅ File received: {target_uri}")
            break
        if not target_uri:
            print("❌ No file uploaded. Stopping.")
            return

    elif source_type == "Magnet":
        if not magnet_link:
            print("❌ Error: You selected 'Magnet' but the link is empty.")
            return
        target_uri = magnet_link

    # Get Trackers
    tracker_list = get_trackers()

    # Construct Aria2 Command
    cmd = [
        "aria2c",
        f"--dir={base_dir}",
        f"--bt-tracker={tracker_list}",
        "--file-allocation=none",
        "--seed-time=0",
        "--summary-interval=1",
        "-x", "16",
        "-s", "16",
        target_uri
    ]

    print(f"\n🚀 Starting Download to: {base_dir}")
    print("⏳ Initializing... (Metadata exchange may take a moment)\n")

    # Run Aria2
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )

    log_file_path = "log.txt"
    print(f"📝 Full logs are being saved to: {os.path.abspath(log_file_path)}")

    with open(log_file_path, "w") as log_file:
        for line in process.stdout:
            log_file.write(line)

            # Refresh Display Logic
            if "*** Download Progress Summary" in line:
                clear_output(wait=True)
                print(f"🚀 Download in progress... (Full logs in {log_file_path})")
                print(f"📂 Saving to: {base_dir}")
                print("-" * 60)
                print(line, end="")
            else:
                print(line, end="")

    process.wait()

    # --- PATCH 3: AUTO-STOP (No Loop) ---
    print("\n✅ Download Process Finished.")
    print("🏁 Cell stopped. You can run this cell again for the next file.")

if __name__ == "__main__":
    run_downloader()